# Social facilitation of laughter and smiles in preschool children
## Data analysis 

#### Caspar Addyman, Charlotte Folquist, Lenka Levakova, Sarah Rees
#### Birkbeck University of London

#### Abstract

Abstract
======
Surprisingly little research has investigated the social dimension of laughter in preschool children.  This experiment studied children’s responses to amusing video clips in the presence or absence of peers. The sample consisted of 9 boys and 11 girls aged 31-49 months (mean 39.8, SD 4.2) who watched three cartoons under three different conditions: individually, in pairs, or in groups of 6 or 8. The social viewing conditions showed significantly higher numbers of laughs and smiles than the individual viewing condition. On average children laughed eight times more in company as on their own, and smiled almost three times as much. No differences were found between pairs and groups, and no association was found between subjective funniness ratings and group size. This suggests that the presence of even a single social partner can change behaviour in response to humorous material. It supports the idea that laughter and smiles are primarily flexible social signals rather than reflexive responses to humour.


In [ ]:
# necessary packages
install.packages(c('readr','reshape2','ez','effects','ggplot2','ggsignif', 'Rmisc', 'svglite','pwr','sjmisc'), repos='http://cran.us.r-project.org')
library(tidyverse)
library(readxl) 
library(ggplot2)
library(ggsignif)
library(reshape2)
library(ez)
library(effects)
library(Rmisc)
library(svglite)
library(pwr)
library(sjPlot)
library(sjmisc)
library()

## Power Calculations

Some useful information found here
http://r-video-tutorial.blogspot.co.uk/2017/07/power-analysis-and-sample-size.html

In [ ]:
#first write a small helper function to calculate partial eta squared..
eta2_p <- function(dfeffect,dferror,FVAL) {
  #If we have a reported effect
  # F(dfeffect,dferror) = FVAL
  # then partial eta square is calculated as
  
  eta = (FVAL * dfeffect) / (FVAL * dfeffect + dferror) 
  # see Lakens(2013) doi:10.3389/fpsyg.2013.00863
  return(eta)
}

We calculate the power based on the effects found in the original Chapman (1973) study of social laughter in 7 & 8 year olds.


In [ ]:
#What was the effect size in Chapman (1973)?
#For laughter F( 2,74) = 27.86, p < .005]
EtaSQ = eta2_p(2,74,27.86) # returns 0.4295405

#For smiling [ F( 2,74 ) = 49.43
eta2_p(2,74,49.43) # returns 0.5719079

##To calculate the group size we convert etasq into f squared aka f2
Chapmanf2 = EtaSQ / (1-EtaSQ) 

##the pwr package lets us calculate the group size 
pwr.f2.test(u=2, f2=Chapmanf2, sig.level=0.05,power = .80)

#this gives v = 13.2 which implies a mimumum group size
# of 17 (so that degrees of freedom is greater than 13.2)
#alternatively we have chosen a group size of 20 then our predicted power is
pwr.f2.test(u=2, v=17, f2=Chapmanf2, sig.level=0.05)

#If we were to use a more conservative general large effect size
# suggested by Cohen (1988) as provided by the cohen.ES function
# then we would need a larger sample
cohenLarge = cohen.ES(test="f2",size="large")$effect.size
pwr.f2.test(u=2, f2=cohenLarge, sig.level=0.05,power = .80)

## Load in the data
First load in the raw data from the spreadsheet. 
Note, to make this work on your machine you will need to set the working directory appropriatelty with `setwd`

In [ ]:
#load raw data
setwd('C:\\Users\\cas\\Dropbox\\projects\\Baby Laughter\\SocialTV')
alldata<- read_excel('SocialTV.xlsx', 'RatingData')

#specify which variables are factors
alldata$SEX <- factor(alldata$SEX)
alldata$ORDER <- factor(alldata$ORDER)
#have a quick look at the data 
head(alldata)

## Basic descriptive stats & correlations

In [ ]:
##############################
# descriptive stats & correlations
##############################

#Descriptive stats
mean(alldata$`Laughs Groups`)
sd(alldata$`Laughs Groups`)
mean(alldata$`Laughs Pairs`)
sd(alldata$`Laughs Pairs`)
mean(alldata$`Laughs Indiv`)
sd(alldata$`Laughs Indiv`)

# we use sjmisc to output descriptive stats table
descr(alldata[,c(3,6:11)],out='txt')

# use sjPlot to output a nicely formatted correlation table.
sjt.corr(alldata[,c(3,6:11)],p.numeric = TRUE, corr.method = "pearson")

## Some helper functions to display the graphs

In [ ]:
##Social TV paper Addyman et al 2017
##Some helper functions to display graphs.

#' Takes the wide (1 row per person) version of the data and makes a long 
#' subset with a single DV (1 col for grouping variable, 1 col for DV)
#' In output grouping variable is "groupVar", DV is "value"
#' 
#' @param allData original data
#' @param col1 1st DV column, etc
groupLongData <- function(allData,col1,col2,col3){
  #subset and rearrange laugh data for anova and plotting
  #convert to long format 
  data.long <- melt(allData,
                    id.vars = c("ID","SEX","ORDER"),
                    measure.vars = c(col1, col2,col3),
                    variable.name ="groupVar")
  #indicate the factor variable
  data.long$groupVar <- factor(data.long$groupVar)
  return(data.long)
}

#' Takes three columns from dataframe and compares them in one way Anova
#' while also plotting a box plot with overlayed individual data points.
#' 
#' @param data.Long Long format version of the data.
#' @param varName Name of the grouping Variable.
#' @examples
#' plotGroupData(longLaughs, "Smiles")
plotGroupData <- function (data.long, varName) {
  #plot groups as box plot and individual values
  ylabel<-paste("Number of ", varName,sep = "")
  p<-ggplot(data=data.long,aes(groupVar,value)) +
    geom_boxplot() +
    geom_point(aes(groupVar,value), size=3,position = position_jitter(width = .1) ) +
    geom_signif(comparisons = list(c(2,3),c(1,3),c(1,2)), step_increase = .12, map_signif_level = TRUE, test="t.test", test.args = list(paired = TRUE)) +
    ylab(ylabel) +
    xlab(NULL) +
    theme_bw(base_size =18)
  return(p)
}

#' plots a standard interaction plot for one-way anova but taking care to
#' use standard errors suitable for repeated errors (like SPSS)
#' 
#' @param data.Long Long format version of the data.
#' @param varName Name of the grouping Variable.
#' @examples
#' plotOneWayANOVA(groupLaughter, "Smiles") 
plotOneWayANOVA <- function (longData, varName) {
  #calculate repeated measures standard errors
  laughsummary<-summarySEwithin(longData, measurevar = "value",betweenvars = NULL,withinvars =  "groupVar")
  # Standard error of the mean
  ylabel<-paste("Number of ", varName,sep = "")
  p<-ggplot(laughsummary, aes(x=groupVar, y=value, group=1)) + 
    geom_errorbar(aes(ymin=value-se, ymax=value+se), width=.1) +
    geom_signif(comparisons = list(c(2,3),c(1,3),c(1,2)), step_increase = .12, map_signif_level = TRUE, test="t.test", test.args = list(paired = TRUE)) +
    geom_line() +
    geom_point(size=3) +
    ylab(ylabel) +
    xlab(NULL) +
    theme_bw(base_size =18)
  return(p)
}

## Number of laughs per group

We need to reshape the data before we plot and analyse it. 

In [ ]:
#To visualise the data we need to reorder it. 
longLaughs<-groupLongData(alldata, "Laughs Groups", "Laughs Pairs", "Laughs Indiv")

#plot data in two different ways 
p1<-plotGroupData(longLaughs,"Laughs")
p2<-plotOneWayANOVA(longLaughs,"Laughs")
show(p1)
show(p2)
##show plots side by side
#grid.arrange(p1, p2, ncol=2)


### One Way ANOVA
Now we run the stats. First we run a 3X3 ANOVA with presentation condition as the factor at three levels (Groups, Pairs, Indiv) and order at three levels (A,B,C) and include sex as a factor


In [ ]:
#initial analysis including sex as variable
ezANOVA(longLaughs,
        dv = value,
        wid = ID,
        between = .(ORDER, SEX),
        within = groupVar )

In [ ]:
Next run the analysis from the paper without sex as a factor

In [ ]:
# 3x3 mixed ANOVA from paper 
ezANOVA(longLaughs,
        dv = value,
        wid = ID,
        between = .(ORDER),
        within = groupVar )

### Planned comparisons

In [ ]:
#run pairwise t-tests
t.test(alldata$`Laughs Groups`,alldata$`Laughs Pairs`, paired = TRUE)
t.test(alldata$`Laughs Groups`,alldata$`Laughs Indiv`, paired = TRUE)
t.test(alldata$`Laughs Pairs`, alldata$`Laughs Indiv`, paired = TRUE)

#but we'd like bonferoni corrected p-values
pairwise.t.test(longLaughs$value,longLaughs$groupVar , 
                p.adjust.method =  "bonferroni", 
                paired = TRUE)


## Number of Smiles

Next we run the same analyses for the number of smiles per condition.

In [ ]:
longSmiles<-groupLongData(alldata, "Smiles Groups", "Smiles Pairs", "Smiles Indiv")
plotGroupData(longSmiles,"Smiles")
plotOneWayANOVA(longSmiles,"Smiles")
ezANOVA(longSmiles,
        dv = value,
        wid = ID,
        within = groupVar )
t.test(alldata$`Smiles Groups`,alldata$`Smiles Pairs`, paired = TRUE)
t.test(alldata$`Smiles Groups`,alldata$`Smiles Indiv`, paired = TRUE)
t.test(alldata$`Smiles Pairs`, alldata$`Smiles Indiv`, paired = TRUE)


### Combined number of laughs and smiles


In [ ]:
longBoth<-groupLongData(allData, "SmilesLaughs Groups", "SmilesLaughs Pairs", "SmilesLaughs Indiv")
plotGroupData(longBoth,"Laughs and Smiles")
plotOneWayANOVA(longBoth,"Laughs and Smiles")
ezANOVA(longBoth,
        dv = value,
        wid = ID,
        within = groupVar )


### Ratings of funniness

After each video each child was asked to rate the video as "Very Funny", "Quite Funny" or "Not Funny" with a verbal request and using the following visual rating scale:
<img src="funninessRating.png" height="450" width="260">

We were interested in whether their ratings differed according to the conditions in which they viewed the video or how much they laughed.

In [ ]:
#But do they find it funny?
#Let's look at the ratings?

longRatings<-groupLongData(alldata,"Funniness Groups", "Funniness Pairs", "Funniness Indiv")
#let R know this is categorical data
longRatings$value<-factor(longRatings$value)
#lets collect responses into table (pick the appropriate columns)
tableRatings<-table(longRatings[,4:5])
#and a data frame for plotting
dfsummaryRatings<-as.data.frame(tableRatings)

ggplot(dfsummaryRatings,aes(groupVar,Freq,fill=value)) +
  geom_bar(stat="identity") + 
  ylab("Count") +
  xlab(NULL) +
  theme_bw(base_size =14)


In [ ]:
#is it significant?
chisq.test(tableRatings)

In [ ]:
longRatings<-rename(longRatings,c("value"="Rating"))
#Finally
#are all videos equally funny?
videoRatings<-groupLongData(alldata,"Rating Video1", "Rating Video2", "Rating Video3")
#let R know this is categorical data
videoRatings$value<-factor(videoRatings$value)
#lets collect responses into table (pick the appropriate columns)
tableVideoRatings<-table(videoRatings[,4:5])
#and a data frame for plotting
dfsummaryRatings<-as.data.frame(tableVideoRatings)
ggplot(dfsummaryRatings,aes(groupVar,Freq,fill=value)) +
  geom_bar(stat="identity") + 
  ylab("Count") +
  xlab(NULL) +
  theme_bw(base_size =14)

In [ ]:
#is it significant?
chisq.test(tableVideoRatings)

A reviewer pointed out that chi squared test is overly conservative because our rating scale is nominal and we have a repeated measure with children giving a rating in 3 conditions. Therefore, after some research it seemed like a logistic regression was appropriate. We use the repolr package.

In [ ]:
#if we are being strictly accurate then repolr package is best way to test ordinal rating data with repeated measures!
longRatings$valueNum<-as.numeric(longRatings$value)
longRatings$groupVarNum<-as.numeric(longRatings$groupVar)
longRatings$ORDERNum<-as.numeric(longRatings$ORDER)
funi<-repolr(valueNum~groupVarNum*ORDERNum,subjects = "ID", times=c(1,2,3),data=longRatings,categories =3)
summary.repolr(funi)

Alternatively, we could to see if funniness is a predictor of amount of laughter in an one-way ANOVA.

In [ ]:

#another check look at amount of laughter by funniess rating
laughsvsfunny<-cbind(longRatings,longLaughs$value)
laughsvsfunny["groupVar"]=NULL
laughsvsfunny<-rename(laughsvsfunny,c("Rating"= "groupVar", "longLaughs$value"="value"))


ezANOVA(laughsvsfunny, dv=value,wid =ID, between=groupVar)

p<-plotOneWayANOVA(laughsvsfunny,varName="Laughs")